# Movie Sentiment Analysis with Keras

In [11]:
# uncomment these for Google collab, will have already been installed in local environment 
# if 'pip install -r requirements.txt' has been run
#!pip install nltk
#!pip install --upgrade gensim

import numpy as np
import os
import os.path

from pdb import set_trace
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
import nltk


import glob
from gensim.models import Word2Vec

import time

[nltk_data] Downloading package punkt to /home/jeremie/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
# MacOSX: See https://www.mkyong.com/mac/wget-on-mac-os-x/ for wget
if not os.path.isdir('../aclImdb'):
    if not os.path.isfile('../aclImdb_v1.tar.gz'):
      !wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz 

    if not os.path.isdir('../aclImdb'):  
      !tar -xf aclImdb_v1.tar.gz 

In [13]:
time_beginning_of_notebook = time.time()
SAMPLE_SIZE=3000
positive_sample_file_list = glob.glob(os.path.join('../aclImdb/train/pos', "*.txt"))
positive_sample_file_list = positive_sample_file_list[:SAMPLE_SIZE]

negative_sample_file_list = glob.glob(os.path.join('../aclImdb/train/neg', "*.txt"))
negative_sample_file_list = negative_sample_file_list[:SAMPLE_SIZE]

import re

# load doc into memory
# regex to clean markup elements 
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r', encoding='utf8')
    # read all text
    text = re.sub('<[^>]*>', ' ', file.read())
    #text = file.read()
    # close the file
    file.close()
    return text


In [14]:
import pandas as pd
from sklearn.utils import shuffle
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

df_positives = pd.DataFrame({'reviews':[load_doc(x) for x in positive_sample_file_list], 'sentiment': np.ones(SAMPLE_SIZE)})
df_negatives = pd.DataFrame({'reviews':[load_doc(x) for x in negative_sample_file_list], 'sentiment': np.zeros(SAMPLE_SIZE)})

print("Positive review(s):", df_positives['reviews'][1][:100])
print("Negative review(s):", df_negatives['reviews'][1][:100])

df = pd.concat([df_positives, df_negatives], ignore_index=True)

df = shuffle(df)

X_train, X_test, y_train, y_test = train_test_split(df['reviews'], df['sentiment'], test_size=0.25)


Positive review(s): NYC model Alison Parker (Cristina Raines) rents a room in an old brownstone where she meets a few bi
Negative review(s): The reason the DVD releases of this film are in black and white is because nobody can get their hand


### Logic to compute DAN model

In [58]:
#ML STUDY GROUP
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import numpy as np

class PreProcessor:
    def __init__(self,REVIEWS,REVIEWS_VAL,LABELS,LABELS_VAL,WE_FILE):
        self.reviews = REVIEWS
        self.reviews_val = REVIEWS_VAL
        self.labels = LABELS
        self.labels_val = LABELS_VAL
        self.we_file = WE_FILE

    def tokenize(self):
#         set_trace()
        print(self.reviews[0])

        tokenizer = Tokenizer()
        tokenizer.fit_on_texts(self.reviews)

        self.sequences = tokenizer.texts_to_sequences(self.reviews)
        self.sequences_val = tokenizer.texts_to_sequences(self.reviews_val)

        self.word_index = tokenizer.word_index
        print("Found %s unique tokens" %(len(self.word_index)))

    def make_data(self):
        self.MAX_SEQUENCE_LENGTH=1000
#         self.MAX_SEQUENCE_LENGTH = max([len(self.sequences[i]) for i in range(len(self.sequences))])
        print("self.MAX_SEQUENCE_LENGTH: {}".format(self.MAX_SEQUENCE_LENGTH))

        data = pad_sequences(self.sequences,maxlen=self.MAX_SEQUENCE_LENGTH)
        data_val = pad_sequences(self.sequences_val,maxlen=self.MAX_SEQUENCE_LENGTH)

#         labels_index = {} # labels_index["Henry IV of France"]
#         answers_index = {} # answers_index[0]

#         for i,j in enumerate(answers):
#             labels_index[j] = i
#             answers_index[i] = j

#         labels = np.zeros((len(self.sequences),1))
#         labels_val = np.zeros((len(self.sequences_val),1))

#         for i in range(len(self.sequences)):
#             labels[i] = labels_index[self.reviews_data[i,1]]

#         for i in range(len(self.sequences_val)):
#             labels_val[i] = labels_index[self.val_data[i,1]]

#         labels = to_categorical(labels,num_classes=len(answers))
#         labels_val = to_categorical(labels_val,num_classes=len(answers))

#         print("Shape of data tensor: " +str(data.shape))
#         print("Shape of label tensor: " +str(labels.shape))

        return data, self.labels, data_val, self.labels_val
        
    def get_word_embedding_matrix(self,EMBEDDING_DIM=100):
        embeddings_index = {}

        if self.we_file == "rand":
            return None

        f = open(self.we_file)

        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
        f.close()

        print('Found %s word vectors.' % len(embeddings_index))

        self.embedding_matrix = np.zeros((len(self.word_index)+1, EMBEDDING_DIM))

        for word, i in self.word_index.items():
            embedding_vector = embeddings_index.get(word)
            if embedding_vector is not None:
                # words not found in embedding index will be all-zeros.
                self.embedding_matrix[i] = embedding_vector

        return self.embedding_matrix


In [59]:
import argparse
import numpy as np

from dan.custom_layers import AverageWords, WordDropout
# from dan.preprocess import PreProcessor

from keras.layers import Embedding, Dense, Input, BatchNormalization, Activation, Dropout
from keras.models import Sequential
from keras.optimizers import Adagrad, Adam
from keras import backend as K

from pdb import set_trace

embedding_dim = 300
num_hidden_layers = 3
num_hidden_units = 300
num_epochs = 100
batch_size = 100
dropout_rate = 0.2
word_dropout_rate = 0.3
activation = 'relu'

args = {}
args['We']='data/glove.6B.300d.txt'
args['Wels']='' ### rand or ''
args['model']='dan'  ### nbow OR dan
args['wd']='y'

reviews=X_train.values
reviews_val=X_test.values
labels=y_train.values
labels_val=y_test.values

In [60]:
# reviews_val.values

In [61]:
pp = PreProcessor(reviews,reviews_val,labels,labels_val,args['We'])
pp.tokenize()

reviews,reviews_val,labels,labels_val = pp.make_data()

embedding_matrix = pp.get_word_embedding_matrix(embedding_dim)


As a lover of the surreal (in art and film) I was pleased to discover this film on IFC. It is definitely a keeper. Most of the other reviews tell the general plot (not all correct) so I won't bother to bore anyone with that. The main thing is the alternate worlds concept which is brought on by Ana's impending illness, and the way she manages to link with someone else after being so "alone", and finally with her family, which I believe is still at least a little troubled. It only can be called a horror movie in that it has frightening scenes but is a fantasy (with a little hint of "coming of age" only because Ana is a pre-teen who "hates boys"). I heartily recommend it to those who appreciate the stretch of their imagination.
Found 41209 unique tokens
self.MAX_SEQUENCE_LENGTH: 1000
Found 400000 word vectors.


In [62]:
model = Sequential()

if args['Wels'] == "rand":
    model.add(Embedding(len(pp.word_index) + 1,embedding_dim,input_length=pp.MAX_SEQUENCE_LENGTH,trainable=False))
else:
    model.add(Embedding(len(pp.word_index)+1,embedding_dim,weights=[embedding_matrix],input_length=pp.MAX_SEQUENCE_LENGTH,trainable=False))

if args['wd'] == 'y':
    model.add(WordDropout(word_dropout_rate))
model.add(AverageWords())


In [63]:
pp.MAX_SEQUENCE_LENGTH

1000

In [64]:
if args['model'] == 'dan':
    for i in range(num_hidden_layers):
        model.add(Dense(num_hidden_units))
        model.add(BatchNormalization())
        model.add(Activation(activation))
        model.add(Dropout(dropout_rate))

model.add(Dense(labels.shape[0]))
model.add(BatchNormalization())
model.add(Dropout(dropout_rate))
model.add(Activation('softmax'))

adam = Adam()
model.compile(loss='categorical_crossentropy',optimizer=adam,metrics=['categorical_accuracy'])

model.summary()

model.fit(reviews,labels,batch_size=batch_size,epochs=num_epochs,\
          validation_data=(reviews_val,labels_val))



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 1000, 300)         12363000  
_________________________________________________________________
word_dropout_4 (WordDropout) (None, 1000, 300)         0         
_________________________________________________________________
average_words_4 (AverageWord (None, 300)               0         
_________________________________________________________________
dense_21 (Dense)             (None, 300)               90300     
_________________________________________________________________
batch_normalization_21 (Batc (None, 300)               1200      
_________________________________________________________________
activation_21 (Activation)   (None, 300)               0         
_________________________________________________________________
dropout_21 (Dropout)         (None, 300)               0         
__________

ValueError: Error when checking target: expected activation_24 to have shape (1500,) but got array with shape (1000,)